In [1]:
import sys
sys.path.append('./')
from utils import get_sequences

import os
os.environ["BRT_ENV"] = 'prod'
import json
import random
import brtdevkit
print(brtdevkit.__version__)
brtdevkit.log = 'info'
import os
os.environ['AWS_PROFILE'] = 'default'
USERNAME = 'li.yu'

import ast
import time
import numpy as np
import pandas as pd
from datetime import datetime, timezone

# from brtdevkit.core.db.athena import AthenaClient, Table
# from brtdevkit.data import Image, Dataset
# from aletheia_dataset_creator.dataset_tools.aletheia_dataset_helpers import *
# from aletheia_dataset_creator.config.dataset_config import *

# from jupiterdata.utils.dataset import query_db
from jupiterdata.config.dataset_config import *
from jupiterdata.utils.dataset import *

pd.set_option('display.max_columns', None)

# %matplotlib inline
%load_ext autoreload
%autoreload 2

8.11.2


In [2]:
""" # run this on cmd to activate aws sso: 
eval "$(/home/li.yu/anaconda3/bin/conda shell.bash hook)"
conda activate query
brt-devkit-auth
aws sso login --profile jupiter_prod_engineer-425642425116
"""
import inspect
print(inspect.getmodule(Dataset))
print(LEFT_CAMERAS)
print(ALL_CAMERA_PAIRS_LIST)

<module 'brtdevkit.data.dataset' from '/home/li.yu/anaconda3/envs/query/lib/python3.10/site-packages/brtdevkit/data/dataset.py'>
['front-center-left', 'front-left-left', 'front-right-left', 'side-left-left', 'side-right-left', 'rear-left', 'T01', 'T02', 'T05', 'T06', 'T09', 'T10', 'T13', 'T14', 'I01', 'I02']
[{'front-center-left': 'front-center-right', 'front-left-left': 'front-left-right', 'front-right-left': 'front-right-right', 'side-left-left': 'side-left-right', 'side-right-left': 'side-right-right', 'rear-left': 'rear-right', 'front-center-right': 'front-center-left', 'front-left-right': 'front-left-left', 'front-right-right': 'front-right-left', 'side-left-right': 'side-left-left', 'side-right-right': 'side-right-left', 'rear-right': 'rear-left'}, {'T01': 'T03', 'T02': 'T04', 'T05': 'T07', 'T06': 'T08', 'T09': 'T11', 'T10': 'T12', 'T13': 'T15', 'T14': 'T16', 'I01': 'I03', 'I02': 'I04'}, {'T02': 'T03', 'T06': 'T07', 'T10': 'T11', 'T14': 'T15', 'I02': 'I03'}]


In [3]:
# test_dataset = Dataset.retrieve(id='64ed2657926aefcd654e8269')
test_dataset = Dataset.retrieve(name='halo_buildup_dust_gilroy_low_dust_human_labeled_v4')
print(test_dataset.id)
test_df = test_dataset.to_dataframe()
print(test_df.shape)
if len(test_df) < 100000:
    test_df = test_df.sort_values('collected_on')
    print(test_df.iloc[0].collected_on, test_df.iloc[-1].collected_on)

66df7905429da12e01ab2fe8
(2295, 221)
2024-07-29T19:08:51.517000 2024-08-27T02:56:48.911000


In [32]:
test_df[['id', 'camera_location']].groupby('camera_location').size()
# test_df[['id', 'operation_time']].groupby('operation_time').size()

camera_location
I02     1
T01    13
T02    20
T05     2
T06     8
T09    39
T10    40
T13    20
T14    23
dtype: int64

In [16]:
test_df.robot_name.unique()

array(['frodo2411'], dtype=object)

## Run Databricks Query

In [4]:
query = f"""
SELECT
  IMJ.id,
  IMJ.operation_time,
  IMJ.camera_location,
  IMJ.robot_name,
  IMJ.collected_on,
  IMJ.special_notes,
  IMJ.hard_drive_name,
  IMJ.spark_request__json,
  IMJ.teleop_request__json,
  IMJ.gps_can_data__json,
  IMJ.geohash,
  IMJ.autonomy_state__json
FROM
  mesa_prod.mesa_lake_prod.image_jupiter AS IMJ
WHERE
  IMJ.robot_name = '1RW8410DTPK808012'
  AND IMJ.collected_on > date('2024-10-01')
  AND IMJ.collected_on <= date('2024-10-31')
"""


df = query_db(query)
if len(df) > 0:
    print(df.shape, len(df)//2)
    if len(df) < 100000:
        df = df.sort_values('collected_on')
        df['collected_on_str'] = df['collected_on'].apply(lambda c: str(c)[:-13])
        print(df.iloc[0].collected_on_str, df.iloc[-1].collected_on_str)
        print(df.iloc[0].id, df.iloc[-1].id)

(477200, 11) 238600


In [5]:
special_notes = df.special_notes.unique()
print(len(special_notes))
list(special_notes)

21


[None,
 'jason zaman high voltage lines',
 'Soy Stubble + Light Cattle Manure',
 'Soy stubble + cattle manure',
 'irrigation hut + water tank',
 'irrigation pivot headland',
 'Jason Zaman High voltage Lines',
 'pivot headland pass',
 'pivot headlands',
 'Pivot House',
 'Road Culvert left of tractor',
 'weed spark said was human (8012 on 10/21 @3:53PM)',
 'JF-183 human capture',
 'Continuous capture tall stalks',
 'tall talks / high debris / high dust / hazy / sundown',
 'JF-183 human capture sundown / haze',
 'soy stubble at night',
 'JF-183',
 'tall stalks with blowing residue',
 'JF-183 Human Capture',
 'soy stubble + cattle manure']

In [11]:
def print_time(df):
    df = df.sort_values('collected_on')
    for i in range(0, len(df), len(df)//5):
        print(df.iloc[i].collected_on, df.iloc[i].id, df.iloc[i].special_notes)
print_time(df2)

2024-10-17 18:58:27.482000+00:00 672581e7d97833442ce4e49b tall stalks with blowing residue
2024-10-17 18:59:50.066000+00:00 67257eab68e6173cadb4134f tall stalks with blowing residue
2024-10-17 19:01:23.639000+00:00 67257d3b498de9072472aef0 tall stalks with blowing residue
2024-10-17 19:02:46.223000+00:00 67258286c4b5acae4b72b827 tall stalks with blowing residue
2024-10-17 19:06:15.680000+00:00 67257b8a424c398443dad179 tall stalks with blowing residue


In [19]:
# df2.groupby('special_notes').size()
df.groupby('camera_location').size()

camera_location
T01    175
T02    355
T05    176
T06    351
T09    156
T10    329
T13    155
T14    307
dtype: int64

## Create a dataset

In [22]:
# unlabeled dataset
Dataset.create(
    name="halo_corn_stubble_w_human_8012_oct_sampled",
    description="2004 images selected from halo_human_w_corn_stubble_0812_oct, viewed all sequences",
    kind='image',  # annotation or image
    image_ids=df.id.to_list())

{}

In [41]:
# labeled, stereo dataset
imageids_to_dataset(
    image_ids=df2.id.to_list(),
    dataset_name="halo_human_in_dust_test_negative_partition",
    dataset_description="15398 low dust level images sampled from 3 human in dust test sets, with thresholds at 20% day 20% dawn_dusk 10% night",
    dataset_kind='pixelwise_annotation',  # image or categorical_annotation or pixelwise_annotation
    mode='stereo',  # stereo or mono
    production_dataset=False,
)

Warning 5267 images do not have a corresponding annotation.
Preparing stereo dataframe for {'T01': 'T03', 'T02': 'T04', 'T05': 'T07', 'T06': 'T08', 'T09': 'T11', 'T10': 'T12', 'T13': 'T15', 'T14': 'T16', 'I01': 'I03', 'I02': 'I04'}...
Size of left dataframe: 10131
Size of stereo dataframe: 10124
Preparing stereo dataframe for {'T02': 'T03', 'T06': 'T07', 'T10': 'T11', 'T14': 'T15', 'I02': 'I03'}...
Size of left dataframe: 6460
Size of stereo dataframe: 6452
Sending 10131 annotated_ids for creating dataset
Time taken to prepare data for dataset creation job: 1.57 mins


In [16]:
# unlabeled, stereo dataset
imageids_to_dataset(
    image_ids=df_sampled.id.to_list(),
    dataset_name="halo_human_w_corn_stubble_0812_oct",
    dataset_description="36256 left images sampled from data collected by machine 8012 October",
    dataset_kind='image',  # image or categorical_annotation or pixelwise_annotation
    mode='stereo',  # stereo or mono
    production_dataset=False,
)

Running create dataset with many images, it will take some time, consider using         the Dataset.create API directly or the imageids_df_to_dataset_fast function
Preparing stereo dataframe for {'T01': 'T03', 'T02': 'T04', 'T05': 'T07', 'T06': 'T08', 'T09': 'T11', 'T10': 'T12', 'T13': 'T15', 'T14': 'T16', 'I01': 'I03', 'I02': 'I04'}...
Size of left dataframe: 36256
Size of stereo dataframe: 36221
Preparing stereo dataframe for {'T02': 'T03', 'T06': 'T07', 'T10': 'T11', 'T14': 'T15', 'I02': 'I03'}...
Size of left dataframe: 18067
Size of stereo dataframe: 18053
Sending 75247 image ids for creating dataset
Time taken to prepare data for dataset creation job: 1.22 mins


## Download a dataset

In [27]:
dataset_name = 'halo_manure_data_candidate'
# dataset_dir = os.path.join('/data/jupiter/datasets/dust_datasets', dataset_name)
# dataset_dir = os.path.join('/data2/jupiter/datasets', dataset_name)
# dataset_dir = os.path.join('/data/jupiter/li.yu/data/dust_data_colletion_for_july_1st', dataset_name)
# dataset_dir = os.path.join('/data/jupiter/datasets/image_quality_datasets', dataset_name)
dataset_dir = os.path.join('/data3/jupiter/datasets/model_positive', dataset_name)
os.makedirs(dataset_dir, exist_ok=True)

test_dataset = Dataset.retrieve(name=dataset_name)
test_df = test_dataset.to_dataframe()
test_dataset.download(dataset_dir, df=test_df)

2024-11-04 06:41:29,961 - brtdevkit - INFO - start downloading dataset: 6728d9b6e038b66c0a00ef6c into /data3/jupiter/datasets/model_positive/halo_manure_data_candidate
INFO:brtdevkit:start downloading dataset: 6728d9b6e038b66c0a00ef6c into /data3/jupiter/datasets/model_positive/halo_manure_data_candidate
/home/li.yu/anaconda3/envs/query/lib/python3.10/site-packages/brtdevkit/data/dataset.py:619: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keys_with_save_path = keys_with_save_path.append(
/home/li.yu/anaconda3/envs/query/lib/python3.10/site-packages/brtdevkit/data/dataset.py:619: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keys_with_save_path = keys_with_save_path.append(
100%|██████████| 64378/64378 [35:17<00:00, 30.41it/s]    
2024-11-04 07:17:19,250 - brtdevkit - INFO - finished downloading dataset: 672